In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../src'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

In [29]:
import json
import csv

In [27]:
!wget https://raw.githubusercontent.com/audioset/ontology/master/ontology.json
!wget http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/unbalanced_train_segments.csv

--2019-10-30 15:47:46--  https://raw.githubusercontent.com/audioset/ontology/master/ontology.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 342780 (335K) [text/plain]
Saving to: ‘ontology.json.1’

ontology.json.1     100%[===================>] 334.75K  1.73MB/s    in 0.2s    

2019-10-30 15:47:47 (1.73 MB/s) - ‘ontology.json.1’ saved [342780/342780]

--2019-10-30 15:47:47--  http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/unbalanced_train_segments.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.10.112, 2607:f8b0:4006:811::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.10.112|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 101468408 (97M) [application/octet-stream]


In [10]:
with open("ontology.json") as f:
    ont=json.load(f)

In [11]:
len(ont)

632

In [47]:
histogram={}
with open("unbalanced_train_segments.csv") as f:
    segments=csv.reader(f)
    header=next(segments)
    header=next(segments)
    header=next(segments)
    for line in segments:
#         print(line[3:])
        if len(line)>3:
            line[3]=line[3].strip()[1:]
            line[-1]=line[-1].strip()[:-1]
        for label in line[3:]:
            histogram.setdefault(label,0)
            histogram[label]+=1
        
    

In [62]:
total_count=sum(histogram.values())

In [100]:
tag2labelmappings={
    "Songbird": ['Bird','Owl','Bird vocalization, bird call, bird song',
                     'Pigeon, dove','Coo','Chirp, tweet','Squawk',
                     'Bird flight, flapping wings','Gull, seagull','Chirp tone',
                     'Hoot'],
    "Water Bird":['Duck','Goose','Quack','Frog',"Croak",'Caw'],
    "Insect":   ['Fly, housefly','Insect', 'Bee, wasp, etc.', 'Buzz','Mosquito',"Cricket","Rustle"],
    "Vehicle":  ['Vehicle','Car', 'Engine','Boat, Water vehicle','Train','Rail transport',
                    'Motor vehicle (road)','Railroad car, train wagon',
                    'Motorboat, speedboat', 'Motorcycle','Subway, metro, underground', 
                     'Lawn mower','Electric shaver, electric razor',
                    'Jet engine', 'Light engine (high frequency)','Sewing machine','Power tool'],
    "Aircraft":  ['Engine','Fixed-wing aircraft, airplane','Aircraft engine',
                'Propeller, airscrew','Aircraft', 'Helicopter'],
    "Running Water":["Waterfall","Waves, surf"],
    "Rain":["Rain","Raindrop","Rain on surface",],
    "Cable":["Bang","Slap, smack","Whack, thwack",
             "Smash, crash","Breaking","Knock","Tap",
             "Thump, thud","Whip","Flap","Clip-clop"],
    "Wind":["Wind","Howl"]}

del tag2labelmappings["Vehicle"]
label2tagmappings={}

for tag in tag2labelmappings:
    for label in tag2labelmappings[tag]:
        label2tagmappings.setdefault(label,[])
        label2tagmappings[label].append(tag)


In [101]:
all_val=[]
for val in tag2labelmappings.values():
    all_val.extend(val)


In [102]:
found=[]
for label in ont:
    if label["name"] in all_val:
#         print(label["name"])
        found.append(label["name"])

In [103]:
len(set(found)),len(set(all_val))

(48, 48)

In [104]:
set(all_val).difference(set(found))

set()

In [105]:
label

{'id': '/m/04zc0',
 'name': 'MP3',
 'description': 'A sound which appears to come from a compressed digital file using the MP3 format, discernible by containing nearly imperceptible compression artifacts.',
 'citation_uri': '',
 'positive_examples': [],
 'child_ids': [],
 'restrictions': ['blacklist']}

In [106]:
count=0
hist_label={}
hist_tag={}
for label in ont:
    if label["name"] in all_val:
#         print(label["name"])
        x=histogram.get(label["id"],0)
        hist_label.setdefault(label["name"],0)
        hist_label[label["name"]]+=x
        tag=label2tagmappings[label["name"]][0]
        hist_tag.setdefault(tag,0)
        hist_tag[tag]+=x
        count+=x
        if x ==0:
            print(label["name"])
#         found.append(label["name"])
count, count/total_count

Gull, seagull


(127195, 0.031638878745697985)

In [107]:
# histogram.keys()

In [108]:
a=list(hist_label.items())
a.sort(key=lambda x: x[1],reverse=True)
a

[('Bird', 26514),
 ('Engine', 15870),
 ('Pigeon, dove', 8403),
 ('Wind', 6454),
 ('Bird vocalization, bird call, bird song', 6075),
 ('Aircraft', 5254),
 ('Chirp, tweet', 4503),
 ('Helicopter', 3578),
 ('Tap', 3319),
 ('Clip-clop', 3205),
 ('Rain', 3175),
 ('Fixed-wing aircraft, airplane', 2858),
 ('Insect', 2825),
 ('Coo', 2797),
 ('Rain on surface', 2739),
 ('Waves, surf', 2692),
 ('Duck', 2483),
 ('Quack', 2432),
 ('Bee, wasp, etc.', 1748),
 ('Thump, thud', 1680),
 ('Goose', 1659),
 ('Whack, thwack', 1441),
 ('Waterfall', 1376),
 ('Rustle', 1361),
 ('Smash, crash', 1355),
 ('Fly, housefly', 1308),
 ('Frog', 1125),
 ('Raindrop', 1106),
 ('Aircraft engine', 1035),
 ('Propeller, airscrew', 926),
 ('Slap, smack', 751),
 ('Cricket', 732),
 ('Howl', 718),
 ('Owl', 513),
 ('Caw', 480),
 ('Flap', 349),
 ('Breaking', 346),
 ('Buzz', 330),
 ('Whip', 329),
 ('Croak', 260),
 ('Bang', 240),
 ('Bird flight, flapping wings', 227),
 ('Mosquito', 222),
 ('Knock', 202),
 ('Chirp tone', 111),
 ('Hoot'

In [109]:
# a

In [92]:
b=list(hist_tag.items())
b.sort(key=lambda x: x[1],reverse=True)
b

[('Songbird', 49232),
 ('Aircraft', 29521),
 ('Cable', 13217),
 ('Insect', 8526),
 ('Water Bird', 8439),
 ('Wind', 7172),
 ('Rain', 7020),
 ('Running Water', 4068)]

In [ ]:
other_hist={"Songbird":0.18,
"Water Bird": 0.088,
"Insect": 0.085,
"Running Water":0.094,
"Rain" :0.046,
"Cable": 0.2,
"Wind" :0.29,
"Aircraft" :0.013}

In [122]:
for k in b:
    print(k[0],"{:.2}, {:.2}, {:.3}".format(k[1]/count,other_hist[k[0]],(k[1]/count)/other_hist[k[0]] ))

Songbird 0.39, 0.18, 2.15
Aircraft 0.23, 0.013, 17.9
Cable 0.1, 0.2, 0.52
Insect 0.067, 0.085, 0.789
Water Bird 0.066, 0.088, 0.754
Wind 0.056, 0.29, 0.194
Rain 0.055, 0.046, 1.2
Running Water 0.032, 0.094, 0.34


In [117]:
audioset 2X Songbird, 17X more Aircraft
ours 2X Cable, 5X Wind